In [ ]:
using Printf: @printf
using Random
using Plots

In [ ]:
heaviside(x::Number; thresh::Number=0.) = x-thresh > 0 ? 1 : 0;

In [ ]:
rng = MersenneTwister(100);

In [ ]:
n_inputs = 100
n_steps = 5000
dt = 1e-3
tend = n_steps * dt
@printf("Simulation time: %g ms.", tend * 1e3)

In [ ]:
freq = 5 # [Hz]
prob = freq * dt
inputs = zeros(n_steps, n_inputs)
inputs[rand(rng, Float64, size(inputs)) .< prob] .= 1;
@printf("Total number of inputs spikes: %d.", sum(inputs))

In [ ]:
tau_mem = 10e-3
tau_syn = 5e-3
α       = exp(-dt / tau_syn)
β       = exp(-dt / tau_mem);

In [ ]:
weight_scale = 7 * (1-β)
weights = weight_scale / sqrt(n_inputs) * randn(rng, Float64, (n_inputs,));

In [ ]:
Iinp  = inputs * weights
Isyn  = zeros(n_steps)
Vm    = zeros(n_steps)
θ     = 1.
n_spk = 0
for t in 1 : n_steps-1
    reset = heaviside(Vm[t], thresh=θ)
    Isyn[t+1] = α * Isyn[t] + Iinp[t]
    Vm[t+1]   = β * Vm[t] + Isyn[t] - reset
    if reset > 0
        n_spk += 1
        @printf(".")
        if n_spk % 100 == 0
            @printf("\n")
        end
    end
end
Vm[Vm .> θ] .= 5θ;

In [ ]:
t = (1 : n_steps) * dt
p1 = plot(t, Vm, color=:black, lw=1, label="", ylabel="Vm")
p2 = plot(t, Isyn, color=:black, lw=1, label="", ylabel="I_syn")
p3 = plot(t, Iinp, color=:black, lw=1, label="", xlabel="Time (s)", ylabel="I_inp")
plot(p1, p2, p3, layout=(3,1), size=(400,400))